In [1]:
from PIL import Image, ImageOps
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

# Check if MPS is available
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class MagicBrush():
    def __init__(self, weight="vinesmsuic/magicbrush-jul7"):
        self.pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
                        weight, 
                        torch_dtype=torch.float16
                    ).to(device)
        self.pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(self.pipe.scheduler.config)
        
    def infer_one_image(self, src_image, instruct_prompt, seed):
        generator = torch.manual_seed(seed)
        image = self.pipe(instruct_prompt, image=src_image, num_inference_steps=20, image_guidance_scale=1.5, guidance_scale=7, generator=generator).images[0]
        return image
        
def load_local_image(image_path):
    image = PIL.Image.open(image_path)
    image = PIL.ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    return image

model = MagicBrush()
image_output = model.infer_one_image(image, prompt, 42)
image_output.show()


/opt/anaconda3/lib/python3.11/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
safety_checker/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /Users/petrsushko/.cache/huggingface/hub/models--vinesmsuic--magicbrush-jul7/snapshots/d75a9f18439f67dbdc0ab113036f491ce2256c53/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /Users/petrsushko/.cache/huggingface/hub/models--vinesmsuic--magicbrush-jul7/snapshots/d75a9f18439f67dbdc0ab113036f491ce2256c53/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /Users/petrsushko/.cache/huggingface/hub/models--vinesmsuic--magicbrush-jul7/snapshots/d75a9f18439f67dbdc0ab113036f491ce2256c53/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /Users/petrsus

  0%|          | 0/20 [00:00<?, ?it/s]

In [25]:
import PIL
def load_local_image(image_path, max_size=(512, 512)):
    image = Image.open(image_path)
    image = ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    image.thumbnail(max_size, Image.Resampling.LANCZOS)  # Resize the image using LANCZOS
    return image
    
local_image_path = "/Users/petrsushko/Desktop/local_photobench/clean/benchmarks/benchmark_images/input/jsxk84.jpeg"

image = load_local_image(local_image_path)
prompt = "Could someone remove the white text and the emoji from this image?"
image_output = model.infer_one_image(image, prompt, 41)
image_output.show()


  0%|          | 0/20 [00:00<?, ?it/s]

In [24]:
image_output.save("/Users/petrsushko/Desktop/local_photobench/clean/benchmarks/benchmark_images/magic_brush_output/jsxk84.jpeg")